# Case Study 2022 - Group 3

In [24]:
from importlib import reload 
import augmentData
import loadAndStoreData
import processData
import drawImages
reload(augmentData)
reload(loadAndStoreData)
reload(processData)
reload(drawImages)

<module 'drawImages' from '/Users/alex/Documents/GitHub/da2-group-3/drawImages.py'>

## Data Augmentation

In this section, the training data is augmented. The function allows to choose the classes for which the augmentations should be done.
It also allows to define the augmentation techniques that are used. 

For each augmentation technique a new subfolder is created. Each subfolder contains the augmented images of the classes chosen.
Depending on the augmentation techniques chosen, this process may neeed a minute or two.

In [4]:
augmentData.performDataAugmentation(
    directory="training_patches/", 
    categories=["ponds", "pools","solar","trampoline"], 
    augmentations=["rotate_images", "move_images", "zoom_images", "change_brightness", "combine_augmentations"]
)


Directory  training_patches_brightnessdown/background  Created 
Directory  training_patches_brightnessdown/background  already exists
Directory  training_patches_brightnessdown/solar  Created 
Directory  training_patches_brightnessdown/solar  already exists
Directory  training_patches_brightnessdown/ponds  Created 
Directory  training_patches_brightnessdown/ponds  already exists
Directory  training_patches_brightnessdown/trampoline  Created 
Directory  training_patches_brightnessdown/trampoline  already exists
Directory  training_patches_brightnessdown/pools  Created 
Directory  training_patches_brightnessdown/pools  already exists
Directory  training_patches_brightnessup/background  already exists
Directory  training_patches_brightnessup/background  already exists
Directory  training_patches_brightnessup/solar  Created 
Directory  training_patches_brightnessup/solar  already exists
Directory  training_patches_brightnessup/ponds  Created 
Directory  training_patches_brightnessup/ponds 

## Data Loading

This section loads the training patches into a numpy array and creates the corresponding label vector.
The result are X_train, X_val, y_train and y_val. 

The images are converted to RGB values, which is why there are 3 channels in the training data.

The training data sets are of dimension (number_of_instances x height x width x 3 channels). 
The label vectors only have one dimension (number_of_instances).

In [36]:
training_data, labels = loadAndStoreData.loadTrainingDataAndLabels(
    folders=[
        "training_patches/",
        "training_patches_brightnessdown",
        "training_patches_brightnessup",
        "training_patches_combined",
        #"training_patches_down",
        #"training_patches_left",
        #"training_patches_right",
        "training_patches_rotation",
        #"training_patches_up",
        "training_patches_zoom"
    ], 
    subdirectories=["background", "ponds", "pools", "solar", "trampoline"])

Shape of training_data array:  (3316, 256, 256, 3)
Shape of labels array:  (3316,)
Shape of training_data array:  (206, 256, 256, 3)
Shape of labels array:  (206,)
Shape of training_data array:  (206, 256, 256, 3)
Shape of labels array:  (206,)
Shape of training_data array:  (206, 256, 256, 3)
Shape of labels array:  (206,)
Shape of training_data array:  (618, 256, 256, 3)
Shape of labels array:  (618,)
Shape of training_data array:  (206, 256, 256, 3)
Shape of labels array:  (206,)
Shape of final training_data:  (4758, 256, 256, 3)
Shape of final labels:  (4758,)


In [37]:
from sklearn.model_selection import train_test_split

labels_categorical = processData.labels_to_categorical(labels)
X_train, X_val, y_train, y_val = train_test_split(training_data, labels_categorical, test_size=0.33, random_state=1, stratify=labels)
print(X_train.shape)
print(y_train.shape)

(3187, 256, 256, 3)
(3187,)


## Model Training 

In [38]:
y_train_encoded = processData.encodeLabels(y_train)

In [39]:
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D
from tensorflow import keras

model = keras.models.Sequential()
model.add(InputLayer(input_shape=(256,256,3)))
model.add(Conv2D(filters=10, kernel_size=(3,3), strides=1, padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(5, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 10)      280       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 10)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 163840)            0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 163840)            655360    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               41943296  
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)              

In [40]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


history = model.fit(X_train, 
                    y_train_encoded, 
                    epochs=20,
                    batch_size=64,
                    validation_split=0.1,
                   )

Epoch 1/20
45/45 [==============================] - 36s 763ms/step - loss: 1.8317 - accuracy: 0.3128 - val_loss: 10.0967 - val_accuracy: 0.0690
Epoch 2/20
45/45 [==============================] - 76s 2s/step - loss: 1.1943 - accuracy: 0.5858 - val_loss: 1.1051 - val_accuracy: 0.7492
Epoch 3/20
45/45 [==============================] - 57s 1s/step - loss: 0.7636 - accuracy: 0.7671 - val_loss: 0.7684 - val_accuracy: 0.8056
Epoch 4/20
45/45 [==============================] - 36s 803ms/step - loss: 0.5483 - accuracy: 0.8347 - val_loss: 0.5485 - val_accuracy: 0.8370
Epoch 5/20
45/45 [==============================] - 43s 949ms/step - loss: 0.4055 - accuracy: 0.8787 - val_loss: 0.7833 - val_accuracy: 0.8025
Epoch 6/20
45/45 [==============================] - 47s 1s/step - loss: 0.3406 - accuracy: 0.8992 - val_loss: 0.6002 - val_accuracy: 0.8276
Epoch 7/20
45/45 [==============================] - 39s 867ms/step - loss: 0.2992 - accuracy: 0.9118 - val_loss: 0.6473 - val_accuracy: 0.8433
Epoch 8

In [41]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np

preds = model.predict(X_val)
preds_argmaxed = np.apply_along_axis(np.argmax, 1, preds)
f1_score(y_val,preds_argmaxed, average='macro'), accuracy_score(y_val, preds_argmaxed) 

(0.6947717994034828, 0.8828771483131763)

In [42]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, preds_argmaxed)

array([[1025,    0,    0,    0,    2],
       [   8,    6,    3,    0,    7],
       [   8,    0,   33,    2,   10],
       [  39,    0,    2,   43,   13],
       [  87,    0,    0,    3,  280]])

## Create predictions

In [47]:
processData.makePredictions("validation_images", convnet=model, stepSize=64, windowSize=(256,256))


Creating predictions for file:  validation_images/UDPYYD.png
Starting sliding window to create patches of size:  256 x 256 .
Still processing, reached patch 10000
Execution time for the last 10.000 patches:  0.014511823654174805  seconds.
Processing continues...
Finished preprocessing of the patches.
Running patches through the Convnet...
Finished predictions, execution time:  41.84871816635132  seconds.

Saved predictions for file:  validation_images/UDPYYD.png 

Elapsed time:  44.858054876327515  seconds.


Creating predictions for file:  validation_images/L7CT2I.png
Starting sliding window to create patches of size:  256 x 256 .
Still processing, reached patch 10000
Execution time for the last 10.000 patches:  0.21338701248168945  seconds.
Processing continues...
Finished preprocessing of the patches.
Running patches through the Convnet...
Finished predictions, execution time:  43.261733055114746  seconds.

Saved predictions for file:  validation_images/L7CT2I.png 

Elapsed time:  

In [17]:
import gc
preprocessed_patches = None
del preprocessed_patches
patch_coordinates = None
del patch_coordinates
X_train = None
del X_train
X_val = None 
del X_val
y_train = None
del y_train
y_val = None
training_data = None
del training_data
X_train_preprocessed = None
del X_train_preprocessed
predictions_array = None
del predictions_array
gc.collect()

3503

In [51]:
processData.nonMaxSuppressBoundingBoxes("validation_images/", iou_threshold=0.0, score_threshold=0.5)

Creating suppressed csv for file:  validation_images/DQIMQN_prediction_suppressed.csv ...


KeyError: 'score'

## Draw Images with predictions

In [53]:
drawImages.saveOrPrintImages(path="./validation_images", print_to_output=False, valBoundingBoxes=True,saveImagesPath="./validation_images", thickness=5)

Saving / printing file:  L7CT2I _annotated.jpg
Saving / printing file:  DQIMQN _annotated.jpg


In [52]:
import csv, json, glob, cv2, random, os
from shapely.geometry import Polygon

def calc_performance(gt_path, pred_path, image_name=None, verbose=0):
    ground_truth = []
    predictions = []

    # Create default performance values
    performances = {
        'file': image_name,
        'tp': 0,
        'fn': 0,
        'fp': 0,
        'f1': 0,
    }

    ## Load ground truth
    with open(gt_path) as f:
        reader = csv.DictReader(f)
        for row in reader:
            row = {k: int(row[k]) if k != 'label' else row[k] for k in row.keys()}
            ground_truth.append(row)

    ## load predictions if path exists
    if os.path.exists(pred_path):
        with open(pred_path) as f:
            reader = csv.DictReader(f)
            for row in reader:
                row = {k: int(row[k]) if k != 'label' else row[k] for k in row.keys()}
                predictions.append(row)

    # Number of false positives equals number of left predictions
    performances['fp'] = max(len(predictions) - len(ground_truth), 0)

    for j, gt in enumerate(ground_truth):
        gt_box = Polygon([(gt['y_upper_left'],  gt['x_upper_left']),
                          (gt['y_upper_left'],  gt['x_lower_right']),
                          (gt['y_lower_right'], gt['x_lower_right']),
                          (gt['y_lower_right'], gt['x_upper_left'])])

        if gt_box.area != (256. * 256.):
            print(f'### Warning {j}: false ground truth shape of {gt_box.area} detected in {image_name}!')
            print(gt['y_lower_right'] - gt['y_upper_left'], gt['x_lower_right'] - gt['x_upper_left'])

        best_found_iou = (None, 0.) # (idx, IoU)
        for i, pred in enumerate(predictions):
            if gt['label'] == pred['label']:
                pred_box = Polygon([(pred['y_upper_left'],  pred['x_upper_left']),
                                    (pred['y_upper_left'],  pred['x_lower_right']),
                                    (pred['y_lower_right'], pred['x_lower_right']),
                                    (pred['y_lower_right'], pred['x_upper_left'])])

                if pred_box.area != (256. * 256.):
                    print(f'### Warning {i}: false predicted shape of {pred_box.area} detected in {image_name}!')
                    print(pred['y_lower_right'] - pred['y_upper_left'], pred['x_lower_right'] - pred['x_upper_left'])

                ## Calculate IoU
                next_iou = (gt_box.intersection(pred_box).area + 1) / (gt_box.union(pred_box).area + 1)

                # If the next found IoU is larger than the previous found IoU -> override
                if next_iou > best_found_iou[1]:
                    best_found_iou = (i, next_iou)

        ## Append metric. If IoU is larger 0.5, then its a true positive, else false negative
        if best_found_iou[0] is not None and best_found_iou[1] >= 0.5:
            del predictions[best_found_iou[0]] # Remove prediction from list!
            performances['tp'] += 1 # Increase number of True Positives
            if verbose == 1:
                print(f'Found correct prediction with IoU of {round(best_found_iou[1], 3)} and label {gt["label"]}!')
        else:
            performances['fn'] += 1 # Increase number of False Negatives
            if verbose == 1:
                print(f'Found false prediction with IoU of {round(best_found_iou[1], 3)} and label {gt["label"]}!')

    ## Calculate F1-Score
    performances['f1'] = (performances['tp'] + 1e-8) / \
                                    (performances['tp'] + 0.5 * (performances['fp'] + performances['fn']) + 1e-8)
    return performances

if __name__ == "__main__":
    path = 'validation_images' # Change if needed

    ## Iterate over all validation images
    for image_path in glob.glob(path + '/*.png'):
        image_name = image_path.split('/')[-1]
        gt_path = image_path[:-4] + '_true.csv' # Ground Truth path
        pred_path = image_path[:-4] + '_prediction_suppressed.csv' # Prediction path
        performance = calc_performance(gt_path, pred_path, image_name)
        print(performance)


{'file': 'L7CT2I.png', 'tp': 1, 'fn': 7, 'fp': 4, 'f1': 0.153846155147929}
{'file': 'DQIMQN.png', 'tp': 0, 'fn': 30, 'fp': 0, 'f1': 6.666666662222222e-10}


## 